# Vamos a trabajar un proyecto

## Pushover

Utilizaremos una herramienta para el envío de notificaciones push siendo gratuito. Puedes encontrar le herramienta en [pushover.net](https://pushover.net)
Con esta cuenta gratuita tenemos que generar las APIs necesarias:
- Token de usuario que se obtiene desde la página principal de Pushover
- Token de aplicación que la consigues desde [build](https://pushover.net/apps/build) y crear una aplicación.

Tendrás que agregar a tu archivo `.env`:
1. PUSHOVER_USER="tu token de usuario"
2. PUSHOVER_TOKEN="tu token de aplicación aquí"

In [36]:
import json
import os
import requests

from dotenv import load_dotenv
from openai import OpenAI

from pypdf import PdfReader
import gradio as gr

In [37]:
load_dotenv(override=True)
openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [38]:
# Configuración de pushover
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [39]:
# Función para registrar detalles del usuario en un archivo JSON de pushover
def push(message):
    """
    Envía un mensaje a través de Pushover.
    """
    print(f"Enviando mensaje push: {message}")
    payload = {
        "user": pushover_user,
        "token": pushover_token,
        "message": message
    }
    requests.post(pushover_url, data=payload)

In [40]:
# Enviando un mensaje de prueba push
push("¡Hola! Este es un mensaje de prueba desde Python.")

Enviando mensaje push: ¡Hola! Este es un mensaje de prueba desde Python.


## Creación de funciones para el LLM
Vamos a crear las herramientas para que podamos registrar diferentes acciones del usuario.
En esta nos registrará los detalles del usuario en `record_user_details` o bien si no se ha podido registrar por el `record_unknown_question`.
Queremos trabajar estas dos funciones en herramientas que pueda utilizar nuestros LLMs.
Esto no deja de ser un JSON que podrá leer los LLM para trabajar.

In [41]:
def record_user_details(email: str, name="Nombre no proporcionado", notes="not provided"):
    """
    Registra los detalles del usuario en un archivo JSON.
    Args:
        email (str): Correo electrónico del usuario.
        name (str): Nombre del usuario. Por defecto es "Nombre no proporcionado".
        notes (str): Notas adicionales del usuario. Por defecto es "not provided".
    """
    push(f"Registrando usuario: {name} con email {email} y notas {notes}")
    return {"redorded": 'ok'}


In [42]:
def record_unknown_question(question:str):
    """ Registra una pregunta no respondida en un archivo JSON.
    Args:
        question (str): Pregunta que no ha sido respondida.
    """
    push(f"Registraando pregunta no respondida: {question}")
    return {"redorded": 'ok'}


In [43]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Utilice esta herramienta para registrar que un usuario está intentando ponerse en contacto contigo y proporcionar una dirección de correo electrónico.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "Correo electrónico del usuario"
            },
            "name": {
                "type": "string",
                "description": "Nombre del usuario, si es que lo facilitó"
            },
            "notes": {
                "type": "string",
                "description": "Notas adicionales del usuario sobre la conversación que merezca la pena ser registrada"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    },
}

In [44]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Utilice esta herramienta para registrar una pregunta que no ha sido respondida.",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Pregunta que no ha sido respondida"
            }
        },
        "required": ["question"],
        "additionalProperties": False
    },
}

In [45]:
tools = [
    {
        "type": "function",
        "function": record_user_details_json
    },
    {
        "type": "function",
        "function": record_unknown_question_json
    }
]

In [46]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Utilice esta herramienta para registrar que un usuario está intentando ponerse en contacto contigo y proporcionar una dirección de correo electrónico.',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'Correo electrónico del usuario'},
     'name': {'type': 'string',
      'description': 'Nombre del usuario, si es que lo facilitó'},
     'notes': {'type': 'string',
      'description': 'Notas adicionales del usuario sobre la conversación que merezca la pena ser registrada'}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': 'Utilice esta herramienta para registrar una pregunta que no ha sido respondida.',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': 'Pregunta que no ha sido respon

In [47]:
# Una versión menos optimizada de la función que maneja las llamadas a herramientas
def handle_tool_calls(tool_calls):
    """
    Maneja las llamadas a herramientas y registra los detalles del usuario o preguntas no respondidas.
    """
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.funtion.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Herraimenta de llamada (Tool_calls): {tool_name}", flush=True)

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)
        
        results.append(
            {
                "role": "tool",
                "content": json.dumps(result), 
                "tool_call_id": tool_call.id
            }
        )
    return results

In [48]:
# Nos carga a partir del diccionario nos da la función con el argumento
# globals()["function_name"]("argumento")
globals()["record_unknown_question"]("esta es una pregunta realmente dificil")

Enviando mensaje push: Registraando pregunta no respondida: esta es una pregunta realmente dificil


{'redorded': 'ok'}

In [49]:
# Esta es una forma más elegante de evitar el IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Herramienta llamada: {tool_name}", flush=True)
        #Hemos añadido el globals() para optimizar el código
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [50]:
reader = PdfReader("perfil/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text
print(linkedin)

with open("perfil/yoMismo.txt", "r", encoding='utf-8') as f:
    yomismo = f.read()

name = "Javi Lázaro"

   
Contactar
hola@javilazaro.es
+34 623386177 (Mobile)
pichu_2707@hotmail.com
www.linkedin.com/in/javi-lazaro
(LinkedIn)
javilazaro.es (Blog)
Aptitudes principales
Ciencia de datos
Gestión de proyectos web
Marketing en motores de búsqueda
(SEM)
Languages
English (Professional Working)
Deutsch (Full Professional)
Certifications
Master en Programación Avanzada
en Python para big Data, Hacking y
Machine Learning
React JS: Aprende React JS desde
cero con ejemplos prácticos
Javier Lázaro
Trabajo en proyectos que necesiten un consultor SEO y CRO,
analista y programador en python para web y diferentes análisis de
mercado ▶ Trabajos en LSO (LinkedIn Search Optimization) para tu
perfil o el de tu empresa
Elgoibar, País Vasco / Euskadi, España
Extracto
He realizado diferentes trabajos en diferentes áreas del marketing
digital como el SEO, CRO, LinkedIn ADS o analítica, desde la
creación de herramientas para automatizar procesos repetitivos con
Python o agilizar otros procesos que de manera más 

In [51]:
system_prompt = f"""Estás actuando como {name}. Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con la carrera, los antecedentes, las habilidades y la experiencia de {name}.
Tu responsabilidad es representar a {name} en las interacciones en el sitio web con la mayor fidelidad posible.
Se te proporciona un resumen de los antecedentes y el perfil de LinkedIn de {name} que puedes usar para responder preguntas.
Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web.
Si no sabes la respuesta a alguna pregunta, usa la herramienta record_unknown_question para registrar la pregunta que no pudiste responder, incluso si se trata de algo trivial o no relacionado con tu carrera.
Si el usuario participa en una conversación, intenta que se ponga en contacto por correo electrónico; pídele su correo electrónico y regístralo con la herramienta record_user_details."""

system_prompt += f"\n\n## Resumen:\n{yomismo}\n\n## LinkedIn Perfil:\n{linkedin}\n\n"
system_prompt += f"En este contexto, chatea con el usuario, siempre con el personaje {name}."

In [58]:
def chat(message, history):
    messages = [
        {
            "role": "system", 
            "content": system_prompt
            }
            ] + history + [

        {
            "role": "user", 
            "content": message
            }
        ]
    done = False
    while not done:
        # Aquí llamamos al LLM pasamos el json de la herramienta y la detecta

        response = openai.chat.completions.create(
            model = 'gpt-4o-mini',
            messages = messages,
            tools = tools,
        )

        finish_reason = response.choices[0].finish_reason

        # Si la respuesta es una llamada a una herramienta, la manejamos

        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


Herramienta llamada: record_user_details
Enviando mensaje push: Registrando usuario: Chicaleta con email pichu2707@gmail.com y notas not provided


## Ahora vamos a implementar
Tenemos esto en local y está muy bien, pero queremos implementarlo en la nube para tener un curriculum 2.0 en la re y que puedan conectarse contigo y preguntar directamente desde tu web.
Lo vamos a crear un archivo .py que llamaremos `app.py` y así podríamos incorporarlo en la web que quisiéramos.

Es importante que si has hecho por partes este trabajo y has ido subiendo los cambios a GitHub no tengas ningún archivo de README en este directorio, si lo tienes cambialo a otro o elimínalo directamente.

Los pasos que vamos a seguir son:
1. Visita [huggingface](https://huggingface.co) y crea una cuenta.
2. En el menú Avatar, en la esquina superior derecha, selecciona "Tokens de acceso". Selecciona "Crear nuevo token". Asígnale permisos de escritura.
3. Tome este token y agréguelo a su archivo `.env`: **HF_TOKEN**=hf_xxx. Si no se detecta durante la implementación, consulte la nota a continuación.
4. Desde la carpeta *modulo_1*, introduzca uv run gradio deploy. Si por alguna razón aún le solicita que introduzca su token HF, interrúmpalo con *Ctrl+C* y ejecute `uv run dotenv -f ../.env run -- uv run gradio deploy`, lo que obliga a que todas sus claves se configuren como variables de entorno.
5. Siga sus instrucciones: asígnele el nombre "career_conversation", especifique `app.py`, elija **cpu-basic** como hardware, confirme que es necesario proporcionar secretos, proporcione su clave de API de OpenAI, su usuario y token de Pushover, y confirme que no se permiten las acciones de GitHub. Nota adicional sobre el token HuggingFace